In [ ]:
# run this cell first
import requests
import lib

url = "http://127.0.0.1:11434/api/generate"

### Overview
This notebook demonstrates a range of language models available via Ollama (https://github.com/ollama/ollama).
Ollama spins up a service that downloads and runs models for you via CLI commands or a dedicated REST API (meaning it can be easily integrated into any web app). Hence, this notebook does not use any dependencies except standard python stuff (HTTP Requests).

Which language models can be used depends on their weights' size, since they have to be loaded exhaustively into the system memory. In our case, a Raspberry Pi 5 with 8GB of RAM provides roughly 7GB of memory available to an AI Model. This makes many models with 7bn parameters work - yet not all of them, since their weights' sizes (I came across) vary between 4-15GB.

### Setup
See the repo's tutorial to learn how to spin up the Ollama service

### Available Models
Browse models available for ollama:
https://ollama.com/search

Tested models are:
- `phi3` (3.8b)
- `deepseek-r1` (7b)
- `llama3.2`(3.8b)
- `moondream` (1.8b, multi-modal)
- `codellama` (7b)

--------------------------------------------

### Demos

#### (One-Way) Chat
Currently, there is no implemented way to engage in a full chat with chat history from within Jupyter. So far, you can only receive text based on your last prompt only.
See the repo's documentation in order to learn how to have a more usual chat experience.

In [ ]:
stream = True

data = {
    "model": "llama3.2", # set model name here
    "prompt": "who won the 2022 Champions League in soccer?"
}
result = requests.post(url, json=data, stream=stream)

response_to_prompt = lib.extract_response_stream(result) if stream else lib.extract_responses(result)

#### Discuss Images
Multi-modal models (so far, only `moondream` is tested) can accept more than just text input. In this case we pass a text prompt along with an image to the model. The prompt asks a question about the image.

In [ ]:
stream = True

data = {
    "model": "moondream",
    "prompt": "what life forms can be seen?",
    "images": [lib.image_to_base64("output/cat.png")]
}

# https://github.com/ollama/ollama/issues/7733
result = requests.post(url, json=data, stream=stream)

response_to_image = lib.extract_response_stream(result) if stream else lib.extract_responses(result)